# NumPy 데이터 로드

이 튜토리얼은 NumPy 배열에서 tf.data.Dataset으로 데이터를 로드하는 예제를 제공합니다.

이 예제에서는 .npz 파일에서 MNIST 데이터세트를 로드합니다. 그러나 NumPy 배열의 소스는 중요하지 않습니다.


## 설정

In [8]:
import numpy as np
import tensorflow as tf

### 넘파이 배열을 파일로 저장하기

In [11]:
a = np.array([1,2,3,4,5])
np.save("a", a)  # a.npy
a = None
a = np.load("a.npy")
print(a)

[1 2 3 4 5]


### .npz 파일에서 로드하기

In [12]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

## `tf.data.Dataset`를 사용하여 NumPy 배열 로드하기

예제 배열과 레이블의 해당 배열이 있다고 가정하면, `tf.data.Dataset.from_tensor_slices`에 튜플로 두 배열을 전달하여 `tf.data.Dataset`을 만듭니다.

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

## 데이터세트 사용하기

### 데이터세트 셔플 및 배치

In [14]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

### 모델 빌드 및 훈련

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [16]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 6s 6ms/step - loss: 9.1093 - sparse_categorical_accuracy: 0.8264
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.5871 - sparse_categorical_accuracy: 0.9214
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4328 - sparse_categorical_accuracy: 0.9423
Epoch 4/10
938/938 [==============================] - 4s 5ms/step - loss: 0.3582 - sparse_categorical_accuracy: 0.9515
Epoch 5/10
938/938 [==============================] - 3s 4ms/step - loss: 0.3237 - sparse_categorical_accuracy: 0.9582A: 4s - loss: 0.3513 - sp
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2859 - sparse_categorical_accuracy: 0.9618
Epoch 7/10
938/938 [==============================] - 5s 5ms/step - loss: 0.2428 - sparse_categorical_accuracy: 0.9658
Epoch 8/10
938/938 [==============================] - 5s 6ms/step - loss: 0.2509 - sparse_categorical_accuracy: 0.9700
Epoch 9/10
938/938 [===

In [7]:
model.evaluate(test_dataset)

157/157 [==============================] - 1s 3ms/step - loss: 0.6077 - sparse_categorical_accuracy: 0.9586


[0.607664942741394, 0.9585999846458435]